In [14]:
import pandas as pd 
import numpy as np
import os 
from sklearn.cluster import KMeans,AgglomerativeClustering,DBSCAN,OPTICS,MeanShift
import os, shutil, glob, os.path
from PIL import Image as pil_image
from matplotlib import pyplot as plt

import sklearn

from sklearn.metrics import pairwise_distances
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import davies_bouldin_score
#importing the header files 
import os
import numpy as np
import pandas as pd
import IPython.display as ipd
from sklearn.manifold import TSNE
import time
import sklearn
from sklearn.decomposition import PCA 
import librosa
from sklearn import mixture
from numpy import unique
from numpy import where
from matplotlib import pyplot as plt
import librosa.display
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.model_selection import GridSearchCV
from kneed import DataGenerator, KneeLocator
%matplotlib inline
import os, shutil, glob, os.path
from PIL import Image as pil_image
from matplotlib import pyplot as plt
import tensorflow as tf
import hdbscan
from numpy import load,save
from scipy.spatial import distance
from scipy.cluster import hierarchy
import umap

In [15]:
def load_filepaths():
    imdir_ideology = 'ideology_image_dataset/'
    imdir_muslim='muslim_image_dataset/'
    ideology_files=os.listdir('ideology_image_dataset/')
    muslim_files=os.listdir('muslim_image_dataset/')
    len(ideology_files),len(muslim_files)

    ideology_files_path=[os.path.join(imdir_ideology,file) for file in ideology_files ]
    muslim_files_path=[os.path.join(imdir_muslim,file) for file in muslim_files]
    return ideology_files_path,muslim_files_path

def loadFeatures(filename):
    print("Loading file : ",filename)
    features= load(filename)
    return features
    

In [64]:
def runGridSearch(estimator,params_dict,train_data):
    
    cv = [(slice(None), slice(None))]
    gs = GridSearchCV(estimator=estimator, param_grid=params_dict, scoring=cv_silhouette_scorer, cv=cv, n_jobs=-1)
    gs.fit(train_data)
    try:
        predicted_labels= gs.best_estimator_.labels_
    except:
        predicted_labels=gs.predict(train_data)
    
    print("best estimator is ",gs.best_estimator_)
    return predicted_labels

In [44]:
def cv_silhouette_scorer(estimator, X):
    estimator.fit(X)
    try:
        cluster_labels = estimator.labels_
    except Exception as e:
      #  print(e,estimator)
        cluster_labels=estimator.predict(X)
    num_labels = len(set(cluster_labels))
    no_noise_labels=num_labels
    if(set(cluster_labels).issuperset({-1})):
        n_clustersLen=len(set(cluster_labels))-1
        no_noise_labels=n_clustersLen
    
    num_samples = len(X)
    if num_labels == 1:
        return -1
    elif num_labels==num_samples:
        return -1
    elif no_noise_labels==1:
        return -1
    else:
        return metrics.silhouette_score(X, cluster_labels)



In [17]:
def evaluation_Score(features,y_pred,output_df,model):
    try:
        
        num_labels=len(set(y_pred))
        total_samples=len(y_pred)
        if(num_labels==1 or num_labels==total_samples):
            output_df.loc[model,'silhouette'] =-1
            output_df.loc[model,'calinski'] =-1
            output_df.loc[model,'davies'] =-1
            
        else:
            output_df.loc[model,'silhouette'] =metrics.silhouette_score(features,y_pred)
            output_df.loc[model,'calinski'] =metrics.calinski_harabasz_score(features, y_pred)
            output_df.loc[model,'davies'] =metrics.davies_bouldin_score(features,y_pred)
            
    

    except Exception as e:
        print(e)
        pass
        

        
    
    return output_df
    
    
    
    
   
    

In [18]:
def getEpsilon(train_data):
    neigh = sklearn.neighbors.NearestNeighbors(n_neighbors=4)
    nbrs = neigh.fit(train_data)
    distances, indices = nbrs.kneighbors(train_data)
    distances = np.sort(distances, axis=0)
    distances = distances[:,1]
    y=distances
    x=list(np.arange(0,len(distances)))
    sensitivity = [1,3, 5, 10, 20,40,60,80, 100, 120,150,180,200,250,300,350,400]
    epsilons=[]
    for s in sensitivity:
        try:
            kneedle = KneeLocator(x,y,S=s, curve='convex', direction='increasing')
            epsilon=kneedle.all_elbows_y[0]
            if(len(epsilons)>=1 and epsilons[-1]-epsilon<=0.001):
                print("")
                
            else:    
                epsilons.append(epsilon)
        
        except Exception as e:
            print(e)
            if(len(epsilons)>=1):
                epsilons.append(epsilons[-1]+s/10)
            else:
                epsilons.append(s/10)
    
   # epsilons.append(0.3)
   # epsilons.append(0.5)
   # epsilons.append(0.8)
        
    print(epsilons)
 
    return epsilons

In [60]:


def runModels(train_data,output_df,filename):
   
    print("HDBSCAN")
    
    clusterer = hdbscan.HDBSCAN(min_cluster_size=2,cluster_selection_epsilon= 0.01,min_samples= 1)
    predicted_labels = clusterer.fit_predict(features)
    if(set(predicted_labels).issuperset({-1})):
        n_clustersLen=len(set(predicted_labels))-1
    else:
        n_clustersLen=len(set(predicted_labels))
        
    evaluation_Score(train_data,predicted_labels,output_df,'HDBSCAN')
    output_df.loc['HDBSCAN','n_clusters']=n_clustersLen
          
    
    np.save(filename+'_hdbscan_labels.npy', np.array(predicted_labels))
    print(output_df)
    
   
    
    print("DBSCAN")
    epsilons=getEpsilon(train_data)
    params_dict = {'eps':epsilons,'min_samples':[2,3,4],'metric':['euclidean','manhattan','mahalanobis']}
    predicted_labels=runGridSearch(sklearn.cluster.DBSCAN(),params_dict,train_data)
    
    evaluation_Score(train_data,predicted_labels,output_df,'DBSCAN')
    if(set(predicted_labels).issuperset({-1})):
        n_clustersLen=len(set(predicted_labels))-1
    else:
        n_clustersLen=len(set(predicted_labels))
    output_df.loc['DBSCAN','n_clusters']=n_clustersLen
    print(output_df)
    np.save(filename+'_dbscan_labels.npy', np.array(predicted_labels))
    
    
    print("Mean shift")
    quantiles=[0.2,0.5,0.8,1]
    params_dict={}
    params_dict['bandwidth']=[]
    for quantile in quantiles:
        params_dict['bandwidth'].append(sklearn.cluster.estimate_bandwidth(train_data, quantile=quantile, n_samples=500))

    params_dict['bandwidth'].append(0.2)
    predicted_labels=runGridSearch(sklearn.cluster.MeanShift(),params_dict,train_data)
    evaluation_Score(train_data,predicted_labels,output_df,'Mean-shift')
    if(set(predicted_labels).issuperset({-1})):
        n_clustersLen=len(set(predicted_labels))-1
    else:
        n_clustersLen=len(set(predicted_labels))
    output_df.loc['Mean-shift','n_clusters']=n_clustersLen
    
    np.save(filename+'_mean_shift_labels.npy', np.array(predicted_labels))
            
    print("Optics")

    params_dict = {'min_samples':[3,4,5],'metric':['euclidean','manhattan','mahalanobis']}
    predicted_labels=runGridSearch(sklearn.cluster.OPTICS(),params_dict,train_data)
    if(set(predicted_labels).issuperset({-1})):
        n_clustersLen=len(set(predicted_labels))-1
    else:
        n_clustersLen=len(set(predicted_labels))
    evaluation_Score(train_data,predicted_labels,output_df,'Optics')
    output_df.loc['Optics','n_clusters']=n_clustersLen
    
    print(output_df)
    
    np.save(filename+'_optics_labels.npy', np.array(predicted_labels))
            
    
    
    

    
   
    return output_df




In [61]:
def runClustering(train_data,filename):
   
    
   # output_df = pd.DataFrame(index=['DBSCAN'],columns=['n_clusters','silhouette','calinski','davies'])
    output_df = pd.DataFrame(index=['HDBSCAN''DBSCAN','Mean-shift','Optics'],columns=['n_clusters','silhouette','calinski'])
   
    output_df=runModels(train_data,output_df,filename)
    
 
   
    return output_df
        
    

In [ ]:
if __name__ == "__main__":
    
    ideology_files_path,muslim_files_path=load_filepaths()
    for file in os.listdir('Image_features/')[0:1]:
        if file.endswith('.npy'):
            print(f'Image_features/{file}')
            features=loadFeatures(os.path.join('Image_features',file))
            print("tsne....")
            tsne_transformed=TSNE(n_components=3, n_jobs=-1).fit_transform(features)
            output_df=runClustering(tsne_transformed,f'image-results-updateGrid/{file}-tsne')
            output_df.to_csv(f'image-results-updateGrid/{file}-tsne.csv')
            
            
            
            print("pca....")
            pca_dims = PCA().fit(features)
            cumsum = np.cumsum(pca_dims.explained_variance_ratio_)
            d = np.argmax(cumsum >= 0.95) + 1
            print(d)
            if(d==1):
                d=d+1
            pca_transformed=PCA(n_components=d).fit_transform(features)
            output_df=runClustering(pca_transformed,f'image-results-updateGrid/{file}-pca')
            output_df.to_csv(f'image-results-updateGrid/{file}-pca.csv')
            
            print("umap....")
            reducer = umap.UMAP(random_state=42,n_components=2)
            umap_transformed = reducer.fit_transform(features)
            output_df=runClustering(umap_transformed,f'image-results-updateGrid/{file}-umap')
            output_df.to_csv(f'image-results-updateGrid/{file}-umap.csv')
            
            
            pca_50 = PCA(n_components=50)
            pca_result_50 = pca_50.fit_transform(features)
            tsne_transformed = TSNE(random_state = 42, n_components=3,verbose=0, perplexity=40, n_iter=400).fit_transform(pca_result_50)
            output_df=runClustering(tsne_transformed,f'image-results-updateGrid/{file}-pca-tsne')
            output_df.to_csv(f'image-results-updateGrid/{file}-pca-tsne.csv')
            


                        #output_dfs=runClustering(tsne_transformed,f'image-results-updateGrid{file}')


            
   # pca_transformed=pca_transform(features)
   # tsne_transformed=tsne_transform(features)

#    output_df1,predicted_labels=runClustering(features,file)
 #   print(output_df1)
   # output_df2,predicted_labels=runClustering(pca_transformed,file)
   # print(output_df2)
    #output_df3,predicted_labels=runClustering(tsne_transformed,file)
    #print(output_df3)
    
        